# Part 1: Implementation of Blind Search Algorithm

In [ ]:
from collections import deque

def read_cube(filename):
    with open(filename, 'r') as file:
        cube = []
        for line in file:
            row = line.strip().split()
            row = [0 if cell in ('0', 'S', 'G') else 1 for cell in row]  # Convert to binary matrix
            cube.append(row)
    return cube

def get_neighbors(x, y, cube):
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, Down, Left, Right
    neighbors = []
    for dx, dy in directions:
        nx, ny = x + dx, y + dy
        if 0 <= nx < len(cube) and 0 <= ny < len(cube[0]) and cube[nx][ny] == 0:
            neighbors.append((nx, ny))
    return neighbors

def dfs(cube):
    stack = [(0, 0, [(0, 0)])]  # (x, y, path)
    visited = set()
    while stack:
        x, y, path = stack.pop()
        if (x, y) in visited:
            continue
        visited.add((x, y))
        if x == len(cube) - 1 and y == len(cube[0]) - 1:
            return path 
        for neighbor in get_neighbors(x, y, cube):
            stack.append((*neighbor, path + [neighbor]))
    return -1

def bfs(cube):
    queue = deque([(0, 0, [(0, 0)])])  # (x, y, path)
    visited = set()
    while queue:
        x, y, path = queue.popleft()
        if (x, y) in visited:
            continue
        visited.add((x, y))
        if x == len(cube) - 1 and y == len(cube[0]) - 1:
            return path
        for neighbor in get_neighbors(x, y, cube):
            queue.append((*neighbor, path + [neighbor]))
    return -1

filename = "cube.txt"
cube = read_cube(filename)
print("DFS Path:", dfs(cube))
print("BFS Path:", bfs(cube))


DFS Path: [(0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (1, 4), (2, 4), (2, 5), (3, 5), (4, 5), (4, 4), (5, 4), (6, 4), (6, 5), (6, 6)]
BFS Path: [(0, 0), (0, 1), (0, 2), (1, 2), (1, 3), (1, 4), (2, 4), (2, 5), (3, 5), (4, 5), (4, 4), (5, 4), (6, 4), (6, 5), (6, 6)]


# Part 2: Implementation of A* Search

In [ ]:
import heapq

def read_cube(filename):
    with open(filename, 'r') as file:
        cube = []
        for line in file:
            row = line.strip().split()
            row = [0 if cell == '0' else 1 if cell == '1' else 2 for cell in row]  # Convert to binary matrix
            cube.append(row)
    return cube

def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def get_neighbors(x, y, cube):
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, Down, Left, Right
    neighbors = []
    for dx, dy in directions:
        nx, ny = x + dx, y + dy
        if 0 <= nx < len(cube) and 0 <= ny < len(cube[0]):
            if cube[nx][ny] == 0 or cube[nx][ny] == 2:  # Can move to empty space or short wall
                neighbors.append((nx, ny))
    return neighbors

def a_star(cube):
    start = (0, 0)
    goal = (len(cube) - 1, len(cube[0]) - 1)
    open_set = []
    heapq.heappush(open_set, (0, start))
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}
    short_walls = {start: 0}

    while open_set:
        _, current = heapq.heappop(open_set)

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()
            return path

        for neighbor in get_neighbors(current[0], current[1], cube):
            tentative_g_score = g_score[current] + 1
            tentative_short_walls = short_walls[current] + (1 if cube[neighbor[0]][neighbor[1]] == 2 else 0)

            if neighbor not in g_score or tentative_g_score < g_score[neighbor] or tentative_short_walls < short_walls[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                short_walls[neighbor] = tentative_short_walls
                heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return -1

filename = "cubeAstar.txt"
cube = read_cube(filename)
print("A* Path:", a_star(cube))

A* Path: [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (4, 3), (4, 4), (5, 4), (6, 4), (6, 5), (6, 6)]
